In [95]:
import NLTE.NLTE_model
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import astropy.constants as const

# Test of Radiation solver

In [64]:

nlte_solver = NLTE.NLTE_model.NLTESolver(NLTE.NLTE_model.Environment(T_phot=10000.0))
A, absorbtion_rate, stimulated_emission_rate = nlte_solver.processes[1].get_einstein_rates()
i23S = nlte_solver.states.all_names.index("23S")
i23P = nlte_solver.states.all_names.index("23P")

print("A rates ordered correctly:", A[i23S, i23P] > 0 and A[i23P, i23S] == 0)
print("Absorbtion rates ordered correctly:", absorbtion_rate[i23S, i23P] == 0 and absorbtion_rate[i23P, i23S] > 0)
print("Stimulated rates ordered correctly:", stimulated_emission_rate[i23S, i23P] > 0 and stimulated_emission_rate[i23P, i23S] == 0)
print("g factor correct:", absorbtion_rate[i23P, i23S]/stimulated_emission_rate[i23S, i23P] == 3)

pop_frac = lambda x: x[0]/sum(x)
nlte_solver = NLTE.NLTE_model.NLTESolver(NLTE.NLTE_model.Environment(T_phot=1000.0))
print("Low temperature limit correct: ", 0.99 < pop_frac(nlte_solver.solve(1e6)[1][[i23S, i23P],-1])) # All in lower state
nlte_solver = NLTE.NLTE_model.NLTESolver(NLTE.NLTE_model.Environment(T_phot=10000.0)) # Approaching T->inf
print("High temperature limit correct: ", 0.5 > pop_frac(nlte_solver.solve(1e6)[1][[i23S, i23P],-1]))

A rates ordered correctly: True
Absorbtion rates ordered correctly: True
Stimulated rates ordered correctly: True
g factor correct: True
Low temperature limit correct:  True
High temperature limit correct:  True


# Test of collisional solver

In [120]:
nlte_solver = NLTE.NLTE_model.NLTESolver(NLTE.NLTE_model.Environment())
nlte_solver.environment.n_e=1e30
y = nlte_solver.solve(1e6)[1][:-2,-1]
#print("High density limit correct: ", 0.5 > pop_frac(nlte_solver.solve(1e6)[1][[i23S, i23P],-1]))
p_i = nlte_solver.states.multiplicities * np.exp(-nlte_solver.states.energies / (const.k_B * nlte_solver.environment.T_electrons*u.K))
pop = p_i / sum(p_i) * sum(y)
print("High density limit correct: ", np.allclose(pop_frac(y), pop_frac(pop)))

High density limit correct:  True
